In [2]:
import os
import librosa
import numpy as np
import pickle
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
def augment_audio(y, sr):
    # Time stretching
    if np.random.random() < 0.5:
        rate = np.random.uniform(0.9, 1.1)  # ±10% speed change
        y = librosa.effects.time_stretch(y, rate=rate)

    # Pitch shifting
    if np.random.random() < 0.5:
        n_steps = np.random.randint(-2, 3)  # ±2 semitones
        y = librosa.effects.pitch_shift(y, sr=sr, n_steps=n_steps)

    # Adding noise
    if np.random.random() < 0.5:
        noise = np.random.normal(0, 0.005, y.shape)
        y = y + noise

    return y

def extract_features(file_path, segment_length=2, hop_length=1, sr=22050):
    y, sr = librosa.load(file_path, sr=sr)
    y = augment_audio(y, sr)
    segment_samples = int(segment_length * sr)
    hop_samples = int(hop_length * sr)

    segments = librosa.util.frame(y, frame_length=segment_samples, hop_length=hop_samples)

    mfccs_list = []
    chroma_list = []
    spectral_contrast_list = []
    tonnetz_list = []

    for segment in segments.T:
        mfccs = librosa.feature.mfcc(y=segment, sr=sr, n_mfcc=13).T
        chroma = librosa.feature.chroma_stft(y=segment, sr=sr).T
        spectral_contrast = librosa.feature.spectral_contrast(y=segment, sr=sr).T
        tonnetz = librosa.feature.tonnetz(y=segment, sr=sr).T

        mfccs_list.append(mfccs)
        chroma_list.append(chroma)
        spectral_contrast_list.append(spectral_contrast)
        tonnetz_list.append(tonnetz)

    return np.array(mfccs_list), np.array(chroma_list), np.array(spectral_contrast_list), np.array(tonnetz_list)

def save_features(file_path, feature_save_dir, mfccs, chroma, spectral_contrast, tonnetz):
    base_filename = os.path.splitext(os.path.basename(file_path))[0]
    if not os.path.exists(feature_save_dir):
        os.makedirs(feature_save_dir)
    with open(os.path.join(feature_save_dir, base_filename + '_mfccs.pkl'), 'wb') as f:
        pickle.dump(mfccs, f)
    with open(os.path.join(feature_save_dir, base_filename + '_chroma.pkl'), 'wb') as f:
        pickle.dump(chroma, f)
    with open(os.path.join(feature_save_dir, base_filename + '_spectral_contrast.pkl'), 'wb') as f:
        pickle.dump(spectral_contrast, f)
    with open(os.path.join(feature_save_dir, base_filename + '_tonnetz.pkl'), 'wb') as f:
        pickle.dump(tonnetz, f)

def load_data(data_dir, feature_dir):
    labels = []
    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        if not os.path.isdir(label_dir):
            continue
        label_feature_dir = os.path.join(feature_dir, label)
        if not os.path.exists(label_feature_dir):
            os.makedirs(label_feature_dir)
        for file_name in os.listdir(label_dir):
            file_path = os.path.join(label_dir, file_name)
            if file_path.endswith('.wav'):  # Ensure only audio files are processed
                mfccs, chroma, spectral_contrast, tonnetz = extract_features(file_path)
                save_features(file_path, label_feature_dir, mfccs, chroma, spectral_contrast, tonnetz)
                labels.append(label)
    return labels

def load_features_from_files(feature_dir):
    mfccs_list = []
    chroma_list = []
    spectral_contrast_list = []
    tonnetz_list = []
    labels = []

    for label in os.listdir(feature_dir):
        label_dir = os.path.join(feature_dir, label)
        if not os.path.isdir(label_dir):
            continue
        for file_name in os.listdir(label_dir):
            if not file_name.endswith('_mfccs.pkl'):
                continue
            base_filename = file_name.replace('_mfccs.pkl', '')
            with open(os.path.join(label_dir, base_filename + '_mfccs.pkl'), 'rb') as f:
                mfccs = pickle.load(f)
            with open(os.path.join(label_dir, base_filename + '_chroma.pkl'), 'rb') as f:
                chroma = pickle.load(f)
            with open(os.path.join(label_dir, base_filename + '_spectral_contrast.pkl'), 'rb') as f:
                spectral_contrast = pickle.load(f)
            with open(os.path.join(label_dir, base_filename + '_tonnetz.pkl'), 'rb') as f:
                tonnetz = pickle.load(f)

            mfccs_list.append(mfccs)
            chroma_list.append(chroma)
            spectral_contrast_list.append(spectral_contrast)
            tonnetz_list.append(tonnetz)
            labels.append(label)

    return mfccs_list, chroma_list, spectral_contrast_list, tonnetz_list, labels

def prepare_data(data_dir, feature_dir):
    # Extract and save features
    labels = load_data(data_dir, feature_dir)

    # Load features from saved files
    mfccs_list, chroma_list, spectral_contrast_list, tonnetz_list, labels = load_features_from_files(feature_dir)

    # Pad sequences to ensure uniform input length for each feature type
    X_mfccs = pad_sequences(mfccs_list, padding='post', dtype='float32', value=-1)
    X_chroma = pad_sequences(chroma_list, padding='post', dtype='float32', value=-1)
    X_spectral_contrast = pad_sequences(spectral_contrast_list, padding='post', dtype='float32', value=-1)
    X_tonnetz = pad_sequences(tonnetz_list, padding='post', dtype='float32', value=-1)
    y = np.array(labels)

    # Encode labels
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)

    # Split the data
    X_mfccs_train, X_mfccs_test, y_train, y_test = train_test_split(X_mfccs, y_encoded, test_size=0.2, random_state=42)
    X_chroma_train, X_chroma_test = train_test_split(X_chroma, test_size=0.2, random_state=42)
    X_spectral_contrast_train, X_spectral_contrast_test = train_test_split(X_spectral_contrast, test_size=0.2, random_state=42)
    X_tonnetz_train, X_tonnetz_test = train_test_split(X_tonnetz, test_size=0.2, random_state=42)

    return (X_mfccs_train, X_chroma_train, X_spectral_contrast_train, X_tonnetz_train, y_train), \
           (X_mfccs_test, X_chroma_test, X_spectral_contrast_test, X_tonnetz_test, y_test), le

# Use this function to prepare the data
data_dir = '/content/drive/MyDrive/AUDIOFORRAAG'  # Update this path to your data directory
feature_dir = '/content/drive/MyDrive/RAAG_features'  # Update this path to your feature directory
(X_mfccs_train, X_chroma_train, X_spectral_contrast_train, X_tonnetz_train, y_train), \
(X_mfccs_test, X_chroma_test, X_spectral_contrast_test, X_tonnetz_test, y_test), le = prepare_data(data_dir, feature_dir)